In [13]:
import pandas as pd

In [14]:
from geopy.distance import geodesic

In [15]:
# Define the file paths
input_file_path = r"C:\Users\wb635947\OneDrive - WBG\Jayati Sethi's files - Pakistan GRADES-B\Data\Dataset\admin\emis_data.xlsx"
output_file_path = r"C:\Users\wb635947\OneDrive - WBG\Jayati Sethi's files - Pakistan GRADES-B\Data\Dataset\nearest_girls_school\closest_schools.csv"

# Load the data from the Excel file
df = pd.read_excel(input_file_path)

# Print the column names to verify them
print(df.columns)

Index(['BEMIS', 'SchoolName', 'District', 'Tehsil', 'SubTehsil', 'UC',
       'VillageName', 'SchoolContactNo', 'Location', 'Shift', 'Genderupdated',
       'SchoolLevel', 'GPSY', 'GPSX'],
      dtype='object')


In [16]:
def find_closest_school(school, schools):
    # Filter schools to only those in the same district
    same_district_schools = schools[(schools['District'] == school['District']) & (schools['Genderupdated'].str.lower().isin(['girls','co-education']))]
    
    min_distance = float('inf')
    closest_school = None
    closest_emis = None
    closest_lat = None
    closest_long = None
    closest_district = None

    
    
    for index, other_school in same_district_schools.iterrows():
        if school['BEMIS'] != other_school['BEMIS']:  # Skip the same school
            distance = geodesic((school['GPSY'], school['GPSX']),
                                (other_school['GPSY'], other_school['GPSX'])).km
            if distance < min_distance:
                min_distance = distance
                closest_school = other_school['SchoolName'] 
                closest_emis = other_school['BEMIS']
                closest_lat = other_school['GPSY']
                closest_long = other_school['GPSX']
                closest_district = other_school['District']
    
    return closest_school, closest_emis, closest_lat, closest_long, min_distance, closest_district


In [17]:
# List to store results
closest_schools = []

# Iterate over each school and find its closest school
for index, school in df.iterrows():
    closest_school, closest_emis, closest_lat, closest_long, distance, closest_district = find_closest_school(school, df)
    
    closest_schools.append({
        'District':school['District'],
        'Gender':school['Genderupdated'],
        'School Name': school['SchoolName'],
        'School Emis Code': school['BEMIS'],
        'School Latitude': school['GPSY'],
        'School Longitude': school['GPSX'],
        'Closest School': closest_school,
        'Closest School Emis Code': closest_emis,
        'Closest School Latitude': closest_lat,
        'Closest School Longitude': closest_long,
        'Distance (km)': distance,
        'Closest School District': closest_district
    })

# Convert to DataFrame
closest_schools_df = pd.DataFrame(closest_schools)

In [19]:
closest_schools_df.to_csv(output_file_path, index=False)